In [ ]:
## Contributors: Sanat Deshpande, Ria George. 
## Group No.: 1934
## Topic: AC06


## The script analyzes tweets from selected twitter handles whose tweets are likely to get retweeted. 
## Then, the script arranges the tweets in ascending order of the total interactions.
## This model basically works on the assumption that tweets from an account with many followers are likely to get many interactions.
## The script analyzes tweets over two days since tweets prior to that are less likely to gain any more interactions. 
## The script uses Tweepy for interacting with the Twitter. 

In [32]:
## Importing the libraries

from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API
from tweepy import Cursor
import tweepy
import numpy as np 
import pandas as pd 
import datetime
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 2000)

In [48]:
## Twitter credentials and authenticator

consumer_key = 'cdztWN0E7WEpfGu1TUKvtZuCj'
consumer_secret = '3sp5ZzbMmWaK1hg8z0xqodoIB1n3fMcMtEbk8YMYzEqIjUlFkF'

access_token = '1146062380213760001-Bc46GVBIvYsL2z2CWwEsDyXpkrAge5'
access_token_secret = 'qilZb9ivTXXg3O4FnoWnEkjRtSwrixq6J11HlnStl52HU'

class TwitterAuthenticator():
	def authenticate_twitter_app(self):
		auth = OAuthHandler(consumer_key, consumer_secret)
		auth.set_access_token(access_token, access_token_secret)
		return auth	

## Declare start and end date for tweets which we are going to analyze

startdate = datetime.datetime(2019, 7, 16, 0, 0, 0)
enddate = datetime.datetime(2019, 7, 17, 0, 0, 0)

In [4]:
## Used to gather the tweets and store it in a JSON file

class TwitterClient():


    def __init__(self, twitter_user = None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)

        self.twitter_user = twitter_user

    def get_twitter_client_api(self):
        self.twitter_client = API(self.auth)
        return self.twitter_client

    def get_user_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user).items(num_tweets):
            #print(tweet.text)
            if("RT" not in tweet.text):
                tweets.append(tweet)
        return tweets


In [45]:
## Analyze the content of tweets and prepare a data frame using pandas and return it

class TweetAnalyzer():
    '''
    Analyze and categorize content from tweets
    '''
    def tweets_to_data_frame(self, tweet_list):
        tweet_final_list = []
        for tweet in tweet_list:
            if(tweet.created_at < enddate and tweet.created_at > startdate):
                if("RT" not in tweet.text):
                    tweet_final_list.append(tweet)
        df = pd.DataFrame(data=[tweet.text for tweet in tweet_final_list], columns=['tweets'])

        df['id'] = np.array([tweet.id for tweet in tweet_final_list])
        df['date'] = np.array([tweet.created_at for tweet in tweet_final_list])
        df['likes'] = np.array([tweet.favorite_count for tweet in tweet_final_list])
        df['RT_count'] = np.array([tweet.retweet_count for tweet in tweet_final_list])
        df['interactions'] = df.likes + df.RT_count
        df['screen_name'] = np.array([tweet.user.name for tweet in tweet_final_list])
        return df


In [55]:
## Main method where all the objects are created and output is displayed in the form of a data frame

twitter_client = TwitterClient()
tweet_analyzer = TweetAnalyzer()
api = twitter_client.get_twitter_client_api()
tweet_list = []
political_handles = ['realDonaldTrump', 'BarackObama', 'HillaryClinton', 'narendramodi', 'PMOIndia', 'Pontifex', 'POTUS', 'WhiteHouse', 'SushmaSwaraj']
for i in political_handles:
    tweets = api.user_timeline(screen_name = i, count = 50)
    tweet_list.append(tweets)
flat_list = [item for sublist in tweet_list for item in sublist]
df = tweet_analyzer.tweets_to_data_frame(flat_list)

df.sort_values(by=['interactions'], ascending = False, inplace=True)
df.head(24)



,tweets,id,date,likes,RT_count,interactions,screen_name
5,"Our Country is Free, Beautiful and Very Successful. If you hate our Country, or if you are not happy here, you can leave!",1151103647637487616,2019-07-16 12:17:50,309956,71838,381794,Donald J. Trump
4,Those Tweets were NOT Racist. I don’t have a Racist bone in my body! The so-called vote to be taken is a Democrat c… https://t.co/jAGPz1saBI,1151129281134768128,2019-07-16 13:59:42,189559,40794,230353,Donald J. Trump
9,"The Democrat Congresswomen have been spewing some of the most vile, hateful, and disgusting things ever said by a p… https://t.co/D5wtHor4lB",1151089268535767042,2019-07-16 11:20:42,154177,35410,189587,Donald J. Trump
3,".....Congresswomen, who I truly believe, based on their actions, hate our Country. Get a list of the HORRIBLE thing… https://t.co/mnDj43eq1k",1151129281919102976,2019-07-16 13:59:42,143006,29321,172327,Donald J. Trump
8,".....shouting of the F...word, among many other terrible things, and the petrified Dems run for the hills. Why isn’… https://t.co/RuKd0QnLOL",1151089269538217984,2019-07-16 11:20:42,130799,29012,159811,Donald J. Trump
6,"If you come after the President, the Country, the Flag - he’s going to defend himself. What the squad doesn’t like… https://t.co/Zq8pGewq31",1151099363445358592,2019-07-16 12:00:49,128371,29858,158229,Donald J. Trump
7,“Billionaire Tech Investor Peter Thiel believes Google should be investigated for treason. He accuses Google of wor… https://t.co/OXOb83lfoB,1151095675213553664,2019-07-16 11:46:09,104186,28428,132614,Donald J. Trump
10,“The best economy in our lifetime!” @IngrahamAngle,1150948978218979328,2019-07-16 02:03:14,109016,20261,129277,Donald J. Trump
2,"Kevin McCarthy @GOPLeader, “The President’s Tweets were not Racist. The controversy over the tweets is ALL POLITICS… https://t.co/FLZRSiUqNc",1151147417028497414,2019-07-16 15:11:46,102504,23013,125517,Donald J. Trump
0,"Yesterday, it was my great honor to host our third annual Made in America Showcase at the @WhiteHouse.... https://t.co/mBha7Qprr9",1151251538079301634,2019-07-16 22:05:30,93571,20565,114136,Donald J. Trump
